In [1]:
import torch
torch.cuda.empty_cache()


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from utils_project import generate_csv,create_dataframe_from_xyz_files,create_X_y_from_dataframe


csv_path = "../../data/energies/train.csv"
path_data = "../../data/atoms/train"
df_train=create_dataframe_from_xyz_files(path_data,csv_path)
X=df_train[['positions', 'energy', 'charges']]

qm7 = X.to_dict("list")

#qm7 = fetch_qm7(align=True)
pos = np.array(qm7['positions'])
full_charges = np.array(qm7['charges'])

n_molecules = pos.shape[0]

In [4]:
# class ElementwiseProd(nn.Module):
#     def __init__(self, input_dim, q, k, act='sigmoid'):
#         super().__init__()
#         self.q = q
#         self.k = k
        
#         # Sélection de la fonction d'activation
#         if act == 'sigmoid':
#             self.activation = torch.sigmoid
#         elif act == 'tanh':
#             self.activation = torch.tanh
#         elif act == 'relu':
#             self.activation = F.relu
#         else:
#             raise ValueError(f"Activation '{act}' non supportée.")
        
#         # Création des k couches linéaires
#         self.hidden_layers = nn.ModuleList([
#             nn.Linear(input_dim, q) for _ in range(k)
#         ])

#     def forward(self, x):
#         output = torch.ones(x.size(0), self.q, device=x.device)
#         for layer in self.hidden_layers:
#             out = self.activation(layer(x))
#             output *= out  # Produit élément par élément
#         return output

import torch
import torch.nn as nn
import torch.nn.functional as F

class ElementwiseProd(nn.Module):
    def __init__(self, input_dim, q, k, act='relu'):
        super(ElementwiseProd, self).__init__()
        self.q = q
        self.k = k
        self.act = act.lower()

        # Crée k couches linéaires de sortie q
        self.hidden_layers = nn.ModuleList([
            nn.Linear(input_dim, q) for _ in range(k)
        ])

    def forward(self, x):
        # Initialiser le produit avec des 1
        output = torch.ones((x.size(0), self.q), device=x.device)

        # Appliquer chaque couche suivie de l'activation
        for layer in self.hidden_layers:
            y = layer(x)
            y = self._apply_activation(y)
            output *= y  # Produit élément par élément

        return output

    def _apply_activation(self, x):
        if self.act == 'relu':
            return F.relu(x)
        elif self.act == 'sigmoid':
            return torch.sigmoid(x)
        elif self.act == 'tanh':
            return torch.tanh(x)
        else:
            raise ValueError(f"Activation '{self.act}' non prise en charge.")


In [5]:
from sklearn.base import BaseEstimator, RegressorMixin
import torch
import torch.nn as nn
import torch.optim as optim

class ElementwiseProdRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim=1, q=10, k=3, act='sigmoid', epochs=100, lr=1e-3, verbose=False):
        self.input_dim = input_dim
        self.q = q
        self.k = k
        self.act = act
        self.epochs = epochs
        self.lr = lr
        self.verbose = verbose
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self._build_model()

    def _build_model(self):
        class FullModel(nn.Module):
            def __init__(self, input_dim, q, k, act):
                super().__init__()
                self.core = ElementwiseProd(input_dim, q, k, act)
                self.output = nn.Linear(q, 1)
            
            def forward(self, x):
                x = self.core(x)
                x = self.output(x)
                return x
        
        self.model = FullModel(self.input_dim, self.q, self.k, self.act).to(self.device)

    def fit(self, X, y):
        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.model.parameters(), lr=self.lr)

        self.model.train()
        for epoch in range(self.epochs):
            optimizer.zero_grad()
            output = self.model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            if self.verbose and epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        return self

    def predict(self, X):
        self.model.eval()
        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            output = self.model(X)
        return output.cpu().numpy().flatten()


In [6]:
M, N, O = 64, 64, 64 #192, 128, 96
grille = "64-64-64"
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [8]:
saved_data = torch.load( f'../models_scattering/scattering_outputs_{grille}.pt', map_location=device)
order_0 = saved_data['order_0']
orders_1_and_2 = saved_data['orders_1_and_2']
order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)
target = qm7['energy']



In [9]:
import numpy as np
from sklearn import linear_model, preprocessing, pipeline, model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import joblib

cross_val_folds = 5  

models = [
    ("Ridge Regression with alpha=0.1", linear_model.Ridge(alpha=0.1)),
    ("PyTorch ElementwiseProd", ElementwiseProdRegressor(input_dim=scattering_coef.shape[1], q=1000, k=3, epochs=50, lr=1e-2))

]

results = []

for name, model in models:
    scaler = preprocessing.StandardScaler()
    regressor = pipeline.make_pipeline(scaler, model)

    target_prediction = model_selection.cross_val_predict(regressor, X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    results.append((name, model, MAE, RMSE))

    print('{}: MAE: {}, RMSE: {}'.format(name, MAE, RMSE))

# Trouver le modèle avec le RMSE le plus bas
best_result = min(results, key=lambda x: x[3])
best_model_name, best_model, best_mae, best_rmse = best_result

print(f"Le meilleur modèle est {best_model_name} avec un RMSE de {best_rmse}.")

/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.99365e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.96524e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.97138e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.77525e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-pa

Ridge Regression with alpha=0.1: MAE: 9.699005882773848, RMSE: 11.965793204274958


/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/pipeline

PyTorch ElementwiseProd: MAE: 9.82907639142246, RMSE: 11.964964674167323
Le meilleur modèle est PyTorch ElementwiseProd avec un RMSE de 11.964964674167323.


/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [10]:
model = ElementwiseProdRegressor(input_dim=scattering_coef.shape[1], q = 1000, k = 50, epochs=200, lr =1e-2, verbose=True)

model.fit(scattering_coef, target)
pred = model.predict(scattering_coef)

rmse = np.sqrt(np.mean((pred - target)**2))
print(rmse)

Epoch 0, Loss: 6197.5195
Epoch 10, Loss: 4182.6445
Epoch 20, Loss: 2837.4985
Epoch 30, Loss: 1939.4681
Epoch 40, Loss: 1339.9357
Epoch 50, Loss: 939.6835
Epoch 60, Loss: 672.4719
Epoch 70, Loss: 494.0793
Epoch 80, Loss: 374.9830
Epoch 90, Loss: 295.4735
Epoch 100, Loss: 242.3921
Epoch 110, Loss: 206.9546
Epoch 120, Loss: 183.2963
Epoch 130, Loss: 167.5017
Epoch 140, Loss: 156.9571
Epoch 150, Loss: 149.9175
Epoch 160, Loss: 145.2178
Epoch 170, Loss: 142.0803
Epoch 180, Loss: 139.9856
Epoch 190, Loss: 138.5872
11.732588475766077
